<a href="https://colab.research.google.com/github/dmaxim/ufl-ai/blob/main/landscapes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pathlib

# download training data
train_url = 'https://zenodo.org/record/5512793/files/train.tgz'
train_dir = tf.keras.utils.get_file(origin=train_url, fname='train', untar=True)
train_dir = pathlib.Path(train_dir)

valid_url = 'https://zenodo.org/record/5512793/files/valid.tgz'
valid_dir = tf.keras.utils.get_file(origin=valid_url, fname='valid', untar=True)
valid_dir = pathlib.Path(valid_dir)

train_image_count = len(list(train_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_dir.glob('*/*.jpg')))
print(train_image_count, valid_image_count)

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 image_size=(150, 150),
                                                                 batch_size=32)

print(train_data, valid_data)

42810482/42810482 [==============================] - 17s 0us/step
14034 3000
Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
<_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [3]:
# build the model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 148, 148, 1)       28        
                                                                 
 flatten_1 (Flatten)         (None, 21904)             0         
                                                                 
 dense_1 (Dense)             (None, 6)                 131430    
                                                                 
Total params: 131458 (513.51 KB)
Trainable params: 131458 (513.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
439/439 [==============================] - 20s 20ms/step - loss: 1.7092 - accuracy: 0.3612 - val_loss: 1.4169 - val_accuracy: 0.4423
Ep

In [5]:
# Add drop out layer
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=1, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 148, 148, 1)       28        
                                                                 
 dropout_1 (Dropout)         (None, 148, 148, 1)       0         
                                                                 
 flatten_3 (Flatten)         (None, 21904)             0         
                                                                 
 dense_3 (Dense)             (None, 6)                 131430    
                                                                 
Total params: 131458 (513.51 KB)
Trainable params: 131458 (513.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
439

In [7]:
# Add more filters
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)



Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_5 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_5 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 dropout_3 (Dropout)         (None, 148, 148, 32)      0         
                                                                 
 flatten_5 (Flatten)         (None, 700928)            0         
                                                                 
 dense_5 (Dense)             (None, 6)                 4205574   
                                                                 
Total params: 4206470 (16.05 MB)
Trainable params: 4206470 (16.05 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/20
439

In [8]:
# Add max poool
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(rate=0.9))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 dropout_4 (Dropout)         (None, 74, 74, 32)        0         
                                                                 
 flatten_6 (Flatten)         (None, 175232)            0         
                                                                 
 dense_6 (Dense)             (None, 6)                 1051398   
                                                      

In [10]:
# Add convolution-pooling-drop
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
# Module 1
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(rate=0.9))

# Module 2
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(rate=0.9))

# Module 3
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(rate=0.9))

# Module 4
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Dropout(rate=0.9))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_8 (Rescaling)     (None, 150, 150, 3)       0         
                                                                 
 conv2d_11 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 74, 74, 32)        0         
 g2D)                                                            
                                                                 
 dropout_9 (Dropout)         (None, 74, 74, 32)        0         
                                                                 
 conv2d_12 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                 

In [12]:
# Add convolution-pooling-without dropout
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
# Module 1
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())

# Module 2
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())

# Module 3
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())

# Module 4
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.keras.activations.relu))
model.add(tf.keras.layers.MaxPool2D())

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()


model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_10 (Rescaling)    (None, 150, 150, 3)       0         
                                                                 
 conv2d_19 (Conv2D)          (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 74, 74, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_20 (Conv2D)          (None, 72, 72, 32)        9248      
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 36, 36, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_21 (Conv2D)          (None, 34, 34, 32)      

In [13]:
# Using MobleNet for transfer learning
import tensorflow as tf

# Downlod pre-trained convolution network -include_top false throw away existing decision layers
base_model = tf.keras.applications.MobileNetV2(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False

base_model.summary()


9406464/9406464 [==============================] - 0s 0us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 75, 75, 32)           864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 75, 75, 32)           128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReL

In [16]:
# build complete inference model
model = tf.keras.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(base_model)
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))
model.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(train_data, epochs=20, validation_data=valid_data)


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_13 (Rescaling)    (None, 150, 150, 3)       0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 max_pooling2d_19 (MaxPooli  (None, 2, 2, 1280)        0         
 ng2D)                                                           
                                                                 
 flatten_13 (Flatten)        (None, 5120)              0         
                                                                 
 dense_13 (Dense)            (None, 6)                 30726     
                                                                 
Total params: 2288710 (8.73 MB)
Trainable params: 307

In [17]:
import tensorflow as tf
import pathlib

# download training data
train_url = 'https://zenodo.org/record/5512793/files/train.tgz'
train_dir = tf.keras.utils.get_file(origin=train_url, fname='train', untar=True)
train_dir = pathlib.Path(train_dir)

# download validation data
valid_url = 'https://zenodo.org/record/5512793/files/valid.tgz'
valid_dir = tf.keras.utils.get_file(origin=valid_url, fname='valid', untar=True)
valid_dir = pathlib.Path(valid_dir)

# print number of training and validation images
train_image_count = len(list(train_dir.glob('*/*.jpg')))
valid_image_count = len(list(valid_dir.glob('*/*.jpg')))
print(train_image_count, valid_image_count)

# package images into tensorflow dataset objects
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                 image_size=(150,150),
                                                                 batch_size=32)
# print tensorflow dataset objects
print(train_data, valid_data)

# download pre-trained convolution network
base_model = tf.keras.applications.MobileNetV2(input_shape=(150,150,3),
                                               include_top=False,
                                               weights='imagenet')
# turn off training for the base model
base_model.trainable = False

# build complete inference model
model = tf.keras.Sequential()
model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255, input_shape=[150,150,3]))
model.add(tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'))
model.add(tf.keras.layers.experimental.preprocessing.RandomRotation(0.2))
model.add(base_model)
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.activations.softmax))

model.summary()

# compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# fit model
model.fit(train_data, epochs=20, validation_data=valid_data)

14034 3000
Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


<_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_14 (Rescaling)    (None, 150, 150, 3)       0         
                                                                 
 random_flip (RandomFlip)    (None, 150, 150, 3)       0         
                                                                 
 random_rotation (RandomRot  (None, 150, 150, 3)       0         
 ation)                                                          
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 5, 5, 1280)        2257984   
 tional)     